In [3]:
from pyspark.sql import SparkSession, DataFrame
import os

In [4]:
local=False
if local:
    spark=SparkSession.builder.master("local[4]") \
                  .config('spark.jars.packages', 'org.postgresql:postgresql:42.2.24') \
                  .appName("RemoveDuplicates").getOrCreate()
else:
    spark=SparkSession.builder \
                      .master("k8s://https://kubernetes.default.svc:443") \
                      .appName("RemoveDuplicates1") \
                      # the image of the worker must be the same as the driver image.  
                      .config("spark.kubernetes.container.image","inseefrlab/jupyter-datascience:py3.9.7-spark3.1.2") \
                      # add the driver pod name, to auto-terminate executor pod after driver pod is deleted 
                      .config("spark.kubernetes.driver.pod.name", os.environ['KUBERNETES_POD_NAME'])
                      .config("spark.kubernetes.authenticate.driver.serviceAccountName",os.environ['KUBERNETES_SERVICE_ACCOUNT']) \
                      .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE']) \
                      .config("spark.executor.instances", "4") \
                      .config("spark.executor.memory","8g") \
                      .config('spark.jars.packages','org.postgresql:postgresql:42.2.24') \
                      .getOrCreate()
        


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a6f52caf-57c6-408d-ac69-11edc316e438;1.0
	confs: [default]
	found org.postgresql#postgresql;42.2.24 in central
	found org.checkerframework#checker-qual;3.5.0 in central
:: resolution report :: resolve 176ms :: artifacts dl 3ms
	:: modules in use:
	org.checkerframework#checker-qual;3.5.0 from central in [default]
	org.postgresql#postgresql;42.2.24 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-------------------------------------------

## Client Mode Executor Pod Garbage Collection

If you run your Spark driver in a k8s pod (as in this example), it is highly recommended to set **spark.kubernetes.driver.pod.name to the name of that pod**. When this property is set, the Spark scheduler will deploy the executor pods with an OwnerReference, which in turn will ensure that once the driver pod is deleted from the cluster, all of the application’s executor pods will also be deleted. The driver will look for a pod with the given name in the namespace specified by spark.kubernetes.namespace, and an OwnerReference pointing to that pod will be added to each executor pod’s OwnerReferences list. 

**Be careful to avoid setting the OwnerReference to a pod that is not actually that driver pod, or else the executors may be terminated prematurely when the wrong pod is deleted.**

In this example, the pod who runs this notebook is the driver pod. So in the config we add the following line

``` python
# The driver pod name is the current pod name.
.config("spark.kubernetes.driver.pod.name", os.environ['KUBERNETES_POD_NAME'])

```

In [5]:
! kubectl get pods

I1025 11:59:54.054167    1677 request.go:665] Waited for 1.168526028s due to client-side throttling, not priority and fairness, request: GET:https://kubernetes.default/apis/apiextensions.k8s.io/v1beta1?timeout=32s
NAME                                               READY   STATUS      RESTARTS   AGE
amundsen-elasticsearch-master-0                    1/1     Running     0          5d1h
amundsen-elasticsearch-master-1                    1/1     Running     0          5d1h
amundsen-elasticsearch-master-2                    1/1     Running     0          5d1h
amundsen-test-frontend-5648d65454-ffgpt            1/1     Running     0          5d1h
amundsen-test-metadata-c57c47666-7bjsz             1/1     Running     0          5d1h
amundsen-test-search-5c8455d784-g56vh              1/1     Running     0          5d1h
datahub-datahub-frontend-6ff5494f44-7gcpx          1/1     Running     0          4d20h
datahub-datahub-gms-5b66978446-g2526               1/1     Running     0          4d20h
da

In [6]:
data=["hadoop spark","hadoop flume","spark kafka","hello spark"]
textRdd=spark.sparkContext.parallelize(data)

In [7]:
splitRdd=textRdd.flatMap(lambda item: item.split(" "))


In [8]:
tupleRdd=splitRdd.map(lambda item: (item,1))
    

In [9]:
reduceRdd=tupleRdd.reduceByKey(lambda x,y:x+y)


In [10]:
strRdd=reduceRdd.map(lambda item: f"{item[0]}, {item[1]}")

In [11]:
list=strRdd.collect()
for item in list:
    print(item)


hadoop, 2
spark, 3
hello, 1
flume, 1
kafka, 1


In [ ]:
strRdd.toDebugString()

In [ ]:
spark.sparkContext.stop()